In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost

Mounted at /content/drive
     |████████████████████████████████| 65.7MB 40kB/s 


In [2]:
path = "./drive/My Drive/Ситилинк Хакатон/"
# path = "./drive/My Drive/ivision/"

In [3]:
import os
from torchvision import transforms
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import datetime

# использует GPU при наличии
class LakeModel():
    def __init__(self, models_path):
        self.device = torch.device('cuda' if 
                                   torch.cuda.is_available() else 'cpu')
        self.network = torch.load(os.path.join(models_path, "network.pth"))
        self.network = self.network.to(self.device)
        self.network.eval()
        with open(os.path.join(models_path, "ModernClassifier.pkl"), "rb") as f:
            self.classifier = pickle.load(f)
        with open(os.path.join(models_path, "ModernRegressor.pkl"), "rb") as f:
            self.regressor = pickle.load(f)
        self.transform = transforms.ToTensor()
        self.size = (128, 128)
        self.columns = ['hour', 'year', 'week', 'day',
                        'days_in_month', 'month', 'quarter', 'season',
                        'prediction']
        self.df = None

 
    def date_to_data(self, frame, name="date", suffix=""):
        assert name in frame, "Нужна колонка с датой"
        date = frame[name].dt
        info = pd.concat([date.days_in_month,
                        date.month,
                        date.quarter,
                        date.month % 12 // 3 + 1,
                        date.hour], axis=1)
        info.columns = ["days_in_month", "month",
                        "quarter", "season", "hour"]
        temp = pd.concat([date.isocalendar(),
                        info], axis=1)
        temp.columns = temp.columns + suffix
        frame = pd.concat([frame, temp], axis=1)
        return frame
    
    def predict_network(self, path_):
        predictions = []
        for filename in tqdm(os.listdir(path_)):
            with Image.open(os.path.join(path_, filename)) as image:
                image = image.resize(self.size)
            X = self.transform(image)
            X = X.view(1, *X.size()).to(self.device)
            prediction = self.network(X).argmax(dim=1).item()
            predictions.append([filename, prediction])
        return predictions
    
    def check_anomaly(self, output):
        arr = []
        for name, value in output:
            if pd.to_datetime(name[:-4]).month in [6, 7, 8]:
                arr.append([name, 0])
            else:
                arr.append([name, value])
        return arr

    def prepare(self, path_):
        df = pd.DataFrame(os.listdir(path_), columns=["name"])
        df["time_parsed"] = pd.to_datetime(df["name"].apply(lambda x: x[:-4]))
        df = self.date_to_data(df, name="time_parsed")

        predictions = self.predict_network(path_)
        predictions_df = pd.DataFrame(predictions,
                                      columns=["filename", "prediction"])
        df = df.merge(predictions_df, left_on="name", right_on="filename")
  
        self.df = df
    
    def predict_classification(self, path_):
        self.prepare(path_)
        output = zip(self.df["name"], self.classifier.predict(self.df[self.columns]\
                                                         .astype(np.float32)))
        return pd.DataFrame(self.check_anomaly(output),
                            columns=["filename", "prediction"])
    
    def time2path(self, d_object):
        if (d_object.hour % 2 != 0):
          time_h = round(int(d_object.hour) + int(d_object.minute) / 60)
          
          if (time_h % 2 != 0): time_h -= 1
          if (time_h > 23): time_h = 0
          if (time_h < 0): time_h = 23

          d_object = d_object.replace(hour=time_h)

        return datetime.datetime.strftime(d_object.replace(minute=0, second=0), "%Y%m%d%H%M") + ".jpg"

    def predict_regreassion(self, path_):
        self.prepare(path_)
        output = zip(self.df["name"], self.regressor.predict(self.df[self.columns]\
                                                             .astype(np.float32)))
        
        output = pd.DataFrame(output, columns=["filename", "prediction"])
        output[output["prediction"] < 0] = 0
        output["prediction"] = np.round(output["prediction"], 2)
        output["time"] = pd.to_datetime(output["filename"].apply(lambda x: x[:-4]))
        output["delta"] = output["prediction"].apply(lambda x: datetime.timedelta(hours=x))
        output["prediction"] = output["time"].dt.to_pydatetime() + output["delta"]
        
        output["prediction_photo"] = output["prediction"].apply(lambda x: self.time2path(x))
        
        return output[["filename", "prediction", "prediction_photo"]]

In [5]:
model = LakeModel(path) # путь к моделям

# Предсказываем установился лед на озере или нет

In [8]:
predictions = model.predict_classification(os.path.join(path, "random")) # путь к папке с фото

In [9]:
predictions

,filename,prediction
0,201812180200.jpg,0
1,201812180000.jpg,0
2,201812171400.jpg,0
3,201812171200.jpg,0
4,201812172200.jpg,0
5,201812171600.jpg,0
6,201812171800.jpg,0
7,201812172000.jpg,0
8,201812180600.jpg,0
9,201812180400.jpg,0


# Предсказываем дату и время полного замерзания Петрозаводской губы Онежского озера

In [10]:
predictions = model.predict_regreassion(os.path.join(path, "random")) # путь к папке с фото

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  PerformanceWarning,


In [11]:
predictions

,filename,prediction,prediction_photo
0,201812180200.jpg,2018-12-19 06:10:48,201812190600.jpg
1,201812180000.jpg,2018-12-18 09:54:00,201812181000.jpg
2,201812171400.jpg,2018-12-19 00:11:24,201812190000.jpg
3,201812171200.jpg,2018-12-19 07:02:24,201812190600.jpg
4,201812172200.jpg,2018-12-18 11:23:24,201812181000.jpg
5,201812171600.jpg,2018-12-18 23:34:12,201812180000.jpg
6,201812171800.jpg,2018-12-18 22:27:36,201812182200.jpg
7,201812172000.jpg,2018-12-18 08:54:36,201812180800.jpg
8,201812180600.jpg,2018-12-19 05:39:00,201812190600.jpg
9,201812180400.jpg,2018-12-18 13:52:48,201812181400.jpg
